In [1]:
import os, sys
from keras import backend as K

%matplotlib inline

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
homeDir = os.getcwd()
dataDir = homeDir + "/data/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = homeDir + "/data/models/"
test_path = dataDir + "test"
results_path = dataDir + "/results"

In [3]:
from utils import *
from Vgg16 import Vgg16
from vgg16bn import Vgg16BN

Creating the model by loading previous weights predicting outputs and using them

In [6]:
vgg = Vgg16()

In [7]:
batch_size=64

In [8]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [9]:
vgg.model.add(Dense(2, activation='softmax')) 

In [10]:
vgg.model.load_weights(model_path+'fineTuneLastLayerScore0_6.h5')

In [11]:
# get_batches function transforms images into arrays and the gets them in batches.
batches = vgg.get_batches(train_path, batch_size=batch_size, shuffle = False)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size, shuffle = False)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
test_path = dataDir + "/test"
test_batches = get_batches(test_path,shuffle=False, batch_size = batch_size)

Found 12500 images belonging to 1 classes.


In [13]:
test_labels = vgg.model.predict_generator(test_batches,test_batches.nb_sample)

In [14]:
save_array(model_path + 'test_labels.bc', test_labels)

In [16]:
test_lables = load_array(model_path+'test_labels.bc')

Will need to seprate conv and dense layers

In [17]:
model = vgg.model
layers = model.layers
lastConvIdx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
convLayers = layers[:lastConvIdx+1]
convModel = Sequential(convLayers)
fcLayers = layers[lastConvIdx+1:]

In [ ]:
# Not needed every time
train_features = convModel.predict_generator(batches,batches.nb_sample)
val_features = convModel.predict_generator(val_batches, val_batches.nb_sample)
test_features = convModel.predict_generator(test_batches, test_batches.nb_sample)

In [20]:
#Not needed every time
save_array(model_path + 'train_convlayer_features.bc', train_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)
save_array(model_path + 'test_convlayer_features.bc', test_features)

In [19]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')
test_features = load_array(model_path+'test_convlayer_features.bc')

In [20]:
combined_features = np.concatenate([trn_features,test_features])

In [21]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [22]:
combined_labels = np.concatenate([trn_labels,test_labels])

In [39]:
def new_weights(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]
opt = RMSprop(lr=0.001)
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape = convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096,activation="relu"),
        Dropout(p),        
        Dense(4096, activation='relu'),
        Dropout(p),
        Dense(2, activation='softmax')
    ])
    for l1,l2 in zip(model.layers, fcLayers): l1.set_weights(new_weights(l2,0.5,p))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [40]:
fc_model = get_fc_model(0.5)

In [42]:
fc_model.fit(trn_features, trn_labels, nb_epoch=1, 
             batch_size=64, validation_data=(val_features, val_labels), shuffle =False)

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 34s - loss: 8.0675 - acc: 0.4995 - val_loss: 7.9623 - val_acc: 0.5060


In [26]:
fc_model = get_fc_model(0.5)

In [27]:
fc_model.fit(trn_features, trn_labels, nb_epoch=1, shuffle = False, 
             batch_size=64, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
 6016/23000 [======>.......................] - ETA: 25s - loss: 4.0979 - acc: 0.5931

KeyboardInterrupt: 

In [ ]:
opt = RMSprop(lr=0.000001)
vgg.model.compile(optimizer=opt,
                loss='categorical_crossentropy', metrics=['accuracy'])

### Predictions

In [ ]:
results_path = dataDir + "/results"

In [ ]:
batchnames, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
filenames = batchnames.filenames
isdog = preds[:,1]
isdog = isdog.clip(min=0.005, max=0.995)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_pseudolabelling.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)